In [17]:
!nvidia-smi

Mon Jul  4 17:29:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   54C    P0    38W / 250W |  15963MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [18]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [19]:
%cd /content/drive/My\ Drive/Licenta/GAN/CycleGAN
!ls

/content/drive/My Drive/Licenta/GAN/CycleGAN
CycleGAN_FID.ipynb	      FID_outputs   prev_versions
CycleGAN_IS.ipynb	      mountain.jpg  startegy_comparison.gdoc
cyclegan_resnet_gen.ipynb     ouputs	    test_CycleGAN
cyclegan_resnet_gen_v2.ipynb  photo_jpg


### Import libraries

In [20]:
import glob
import cv2
from PIL import Image
import numpy as np
from scipy import linalg
import tensorflow as tf
from tensorflow import keras

### Define functions needed for computing FID

In [21]:
def compute_distribution_statistics(real_features, fake_features):
  # compute mean
  mu_real, mu_fake = np.mean(real_features, axis=0), np.mean(fake_features, axis=0)
  # compute covariance
  sigma_real, sigma_fake = np.cov(real_features, rowvar=False), np.cov(fake_features, rowvar=False)
  return mu_real, mu_fake, sigma_real, sigma_fake

In [22]:
# calculate frechet inception distance
def frechet_distance(mu_real, mu_fake, sigma_real, sigma_fake):

  # compute the distance between the means
  mean_distance = np.sum((mu_real - mu_fake) ** 2.0)
  
  # compute the matrix square root of the dot product of the two covariance matrices
  covariance_term = linalg.sqrtm(np.dot(sigma_real, sigma_fake)).real
  
  # compute the distance between the standard deviations
  std_distance = np.trace(sigma_real + sigma_fake - 2.0 * covariance_term)

  # return the frechet inception distance
  return mean_distance + std_distance

In [23]:
#@title Choose art style

style = 'VanGogh' #@param ["Cezanne", "Monet", "Ukiyoe", "VanGogh"]

### Read the real and generated images

In [24]:
real_art_paths = glob.glob(f'FID_outputs/{style.lower()}/real_art/*.jpg')
real_art_dataset = np.array([np.array(Image.open(real_art_path)) for real_art_path in real_art_paths])
print(f"Loaded real art dataset of shape = {real_art_dataset.shape}")

generated_art_paths = glob.glob(f'FID_outputs/{style.lower()}/generated_art/*.jpg')
generated_art_dataset = np.array([np.array(Image.open(generated_art_path)) for generated_art_path in generated_art_paths])
print(f"Loaded generated art dataset of shape = {generated_art_dataset.shape}")

Loaded real art dataset of shape = (800, 256, 256, 3)
Loaded generated art dataset of shape = (7038, 256, 256, 3)


### Preprocess data

In [25]:
# convert to 32-bit floats
real_art_dataset = real_art_dataset.astype('float32')
generated_art_dataset = generated_art_dataset.astype('float32')

In [26]:
# resize images in real art dataset to (299, 299, 3)
real_art_dataset = np.asarray([cv2.resize(real_art, (299, 299)) for real_art in real_art_dataset])
# resize images in generated art dataset to (299, 299, 3)
generated_art_dataset = np.asarray([cv2.resize(generated_art, (299, 299)) for generated_art in generated_art_dataset])

In [27]:
# preprocess images in real art dataset so that they can be applied to InceptionV3 model
real_art_dataset = keras.applications.inception_v3.preprocess_input(real_art_dataset)
# preprocess images in generated art dataset so that they can be applied to InceptionV3 model
generated_art_dataset = keras.applications.inception_v3.preprocess_input(generated_art_dataset)

### Load model and get predictions

In [28]:
# load InceptionV3 model for FID
model = keras.applications.inception_v3.InceptionV3(include_top = False, pooling = 'avg', input_shape = (299, 299, 3))

In [29]:
# get features of real and CycleGAN generated art using InceptionV3
real_features = model.predict(real_art_dataset)
fake_features = model.predict(generated_art_dataset)

### Compute FID

In [30]:
# compute distribution statistics for the real and generated examples
mu_real, mu_fake, sigma_real, sigma_fake = compute_distribution_statistics(real_features, fake_features)

In [31]:
# compute FID
FID = frechet_distance(mu_real, mu_fake, sigma_real, sigma_fake)
print(f"Frechet Inception Distance = {FID}")

Frechet Inception Distance = 115.21052746335013


In [32]:
FID/2

57.605263731675066